<a href="https://colab.research.google.com/github/fitomx19/-web-API-with-ASP.NET-Core/blob/master/EntrenamientoSpacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entrenamiento Spacy -> Named Entity Recognition

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


A continuacion se muestran los paquetes que se van a utilizar para que funcione el sistema

In [2]:
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin
import spacy
from utilidades import etiquetador_personas,completar_tabla
nlp = spacy.load("en_core_web_sm")

De un texto que esta probado que no funciona en su totalidad -> vamos a buscar todas las entidades que sean nombres

In [3]:
prueba = "Se viralizó en redes sociales, el caso de una víctima ADOLFO HUERTA donde recibio una estafa dentro de un restaurante de la de la Ciudad de México. De acuerdo con la denuncia en redes sociales, el restaurante Terraza, ubicado en el Zócalo de la Ciudad de México cobró excesivamente a una usuaria que tuvo que pagar más de 4 mil pesos mexicanos por sólo comer tacos, guacamole y unas cervezas."


(Para modificarlo unicamente cambie el texto en la variable llamada **prueba**)


In [4]:
document = nlp(prueba)
colors = {'Personas': "#85C1E9"}
options = {"ents": ['PERSON'], "colors": colors}
spacy.displacy.render(document, style="ent", jupyter=True, options=options) # display in Jupyter

Como podemos ver se observan coloreadas de morado las entidades que hace referencia a persona, pero esto no es correcto, entonces enseñaremos a la maquina a entender cuales son las palabras que si son **PERSONAS**

In [5]:
personas = ["ADOLFO HUERTA"]

In [6]:
valores = completar_tabla(personas,prueba)

ADOLFO HUERTA


In [7]:
def entrenamiento(valores,prueba):
  db = DocBin() # create a DocBin object

  #for x in tqdm(range(len(valores))): # data in previous format
  nlp = spacy.load("en_core_web_sm")    
  doc = nlp.make_doc(prueba) # create doc object from text
  ents = []
  for start, end, label in (valores): # add character indexes
      #print(start)
      #print(end)
      #print(label)
      span = doc.char_span(start, end, label=label, alignment_mode="contract")
      if span is None:
          print("Skipping entity")
      else:
          ents.append(span)
          #print(span)
      doc.ents = ents # label the text with the ents
      print(ents)
      db.add(doc)

  db.to_disk("./train.spacy") # save the docbin object

En esta seccion se creara el archivo de preentrenamiento con las entidades que seleccionamos

In [8]:
entrenamiento(valores,prueba)

[ADOLFO HUERTA]


In [9]:
!python -m spacy init fill-config base_config.cfg config.cfg

2022-11-08 21:08:36.209938: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Usage: python -m spacy init fill-config 
           [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config.cfg' does not exist.


In [10]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy


2022-11-08 21:08:42.221204: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-11-08 21:08:43,306] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-08 21:08:43,328] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-11-08 21:08:43,333] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-08 21:08:43,335] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-11-08 21:08:43,452] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ne

- nlp1 es la base de datos de entrenamiento nueva que nos servira para identificar de mejor manera los valores entrenados

- nlp es la base de spacy standard

In [11]:
nlp1 = spacy.load(r"./output/model-best") #load the best model

Probemos con la base de datos de spacy -->

In [12]:
doc = nlp("ADOLFO HUERTA probablemente la siguiente semana sea llevado a presentar ante el Juez") # input sample text
#doc = nlp("Cada cierto tiempo se menciona que Markov Colopa es un antiguo miembro de nuestra compaÃ±ia de aceleracion de empresas Sonico, por lo tanto Andres Gomez Farias es un inventor asociado a la firma de tecnologia quizas tambien ADOLFO HUERTA nos pueda ayudar") # input sample text
colors = {'Personas': "#85C1E9"}
options = {"ents": ['PERSON'], "colors": colors}
spacy.displacy.render(doc, style="ent", jupyter=True, options=options) # display in Jupyter

Ahora con la nuestra --->

In [13]:
doc = nlp1("ADOLFO HUERTA probablemente la siguiente semana sea llevado a presentar ante el Juez") # input sample text
#doc = nlp1("Cada cierto tiempo se menciona que Markov Colopa es un antiguo miembro de nuestra compaÃ±ia de aceleracion de empresas Sonico, por lo tanto Andres Gomez Farias es un inventor asociado a la firma de tecnologia quizas tambien ADOLFO HUERTA nos pueda ayudar") # input sample text
colors = {'Personas': "#85C1E9"}
options = {"ents": ['PERSON'], "colors": colors}
spacy.displacy.render(doc, style="ent", jupyter=True, options=options) # display in Jupyter

Podemos observar que con nuestro entrenamiento ahora ya se muestra correctamente Adolfo Huerta y se le reconoce como persona.